In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import special
np.set_printoptions(threshold=np.inf)

## Small example for binomial coefficient nCr using scipy

In [2]:
B=special.binom(0, 0) #binom(n,r)
print(B)

1.0


## Find Span

In [3]:
def FindSpan(n_inp,degree_inp,u_inp,knot_vector_inp):
    x=knot_vector_inp[degree_inp+1]
    if (u_inp < x):
        return degree_inp
    else:
        for i,pos in enumerate(knot_vector_inp):
            if math.floor(u_inp) == pos:
                return (i)

## Non Zero Basis Functions

In [4]:
#The output is stored in N[0],....N[p]
#These are non zero functions in a given span
#i is the span of the u value
#we get N[i-p].........,N[i]
# So if i is 3 then N[3] is stored in N[p] of the output matrix
def BasisFuns(i,u,p,U):
    N =np.zeros(p+1)
    N[0] = 1.0
    left =np.zeros(p+2)
    right =np.zeros(p+2)
    for j in range(1,p+1):
        left[j] = u - U[i+1-j]
        right[j] = U[i+j] - u
        saved= 0.0
        for r in range(0,j):
            temp = N[r]/(right[r+1]+left[j-r])
            N[r] = saved + right[r+1]*temp
            saved = left[j-r]*temp
        N[j] = saved
    return N

## Derivatives of Non_Zero basis Functions

In [5]:
# Derivatives are stored in ders[k][j]
# ders[k][j] is the kth derivative of function N_(i-p+j,p)
# If k>p the derivatives are zero.
def DersBasisFuns(i,u,p,m,U):
    #Inititation of dimentions for 2D matrices
    ndu=np.zeros((p+1,p+1))
    ders=np.zeros((p+1,p+1))
    a=np.zeros((2,p+1))
    left =np.zeros(p+2)
    right =np.zeros(p+2)
    
    ndu[0][0]=1.0
    for j in range(1,p+1):
        left[j] = u - U[i+1-j]
        right[j] = U[i+j] - u
        saved=0.0
        for r in range(j):
            #Lower triangle
            ndu[j][r] = right[r+1]+left[j-r]
            temp=ndu[r][j-1]/ndu[j][r]
            #Upper triangle
            ndu[r][j] = saved+(right[r+1]*temp)
            saved=left[j-r]*temp
        ndu[j][j] = saved
    for j in range (p+1): #Load the basis functions
        ders[0][j]=ndu[j][p]
    #This secion computes the derivatives
    for r in range(p+1):
        s1=0
        s2=1 #Alternative rows in array a
        a[0][0] = 1.0
        #Loop to compute kth derivative
        for k in range(1,m+1):
            d=0.0
            rk=r-k
            pk=p-k
            if(r>=k):
                a[s2][0]=a[s1][0]/ndu[pk+1][rk]
                d=a[s2][0]*ndu[rk][pk]
            if(rk>=-1):
                j1=1
            else:
                j1=-rk
            if(r-1<=pk):
                j2=k-1
            else:
                j2=p-r
            for j in range (j1,j2+1):
                a[s2][j] =(a[s1][j]-a[s1][j-1])/ndu[pk+1][rk+j]
                d += (a[s2][j]*ndu[rk+j][pk])
            if(r<=pk):
                a[s2][k]=-a[s1][k-1]/ndu[pk+1][r]
                d+=(a[s2][k]*ndu[r][pk])
            ders[k][r]=d
            #Switch rows
            j=s1
            s1=s2
            s2=j
            #Multiply through by the correct factors
    r=p
    for k in range(1,m+1):
        for j in range(p+1):
            ders[k][j] =ders[k][j]* r
        r =r* (p-k)
    return ders

## Aders

In [6]:
#An extra W is given as input to function for convinience which contain weights of corresponding control points
def SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,u,v,d):
    #SKL=np.zeros((p+1,q+1,3))
    SKL=np.zeros((20,20,3))
    temp=np.zeros((q+1,3))
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,q)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]*P[uspan-p+r][vspan-q+s]*W[uspan-p+r][vspan-q+s]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning Aders here

## wders

In [7]:
def SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,u,v,d):
    #SKL=np.zeros((p+1,q+1,1)) #Have to cross check if this is right way of dimensioning SKL
    SKL=np.zeros((20,20,1))
    temp=np.zeros((q+1,1))    # Have to be because W is a scalar unlike P which is vector
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,q)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    #print('USPAN',uspan)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    #print('VSPAN',vspan)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]*W[uspan-p+r][vspan-q+s]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning wders here

## Only derivatives of NURBS Functions

In [8]:
def SurfaceDerivsAlgNURBS(n,p,U,m,q,V,P,W,u,v,d):
    #SKL=np.zeros((p+1,q+1,1)) #Have to cross check if this is right way of dimensioning SKL
    SKL=np.zeros((20,20,1))
    temp=np.zeros((q+1,1))    # Have to be because W is a scalar unlike P which is vector
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,q)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    #print('USPAN',uspan)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    #print('VSPAN',vspan)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning wders with weights equal to 1 here

## Function to extract derivatives of NURBS Surface

In [9]:
#Functions gives SKL matrix which contain kth ans lth derivatives of the NURBS Surface function S(u,v)
#For example 1st derivative of S(u,v) wrt to xi_para is stored in SKL[1][0]
#Likewise 1st derivative of S(u,v) wrt to eta_para is stored in SKL[0][1]

def RatSurfaceDerivs(Aders,wders,d):
    #SKL=np.zeros((p+1,q+1,3))
    SKL=np.zeros((20,20,3))
    for k in range(d+1):
        for l in range(d-k+1):
            v=Aders[k][l]
            for j in range(1,l+1):
               # print('B',special.binom(l,j))
                v=v-special.binom(l,j)*wders[0][j]*SKL[k][l-j]
            for i in range(1,k+1):
                v=v-special.binom(k,i)*wders[i][0]*SKL[k-i][l]
                v2 = 0.0
                for j in range(1,l+1):
                    v2=v2+special.binom(l,j)*wders[i][j]*SKL[k-i][l-j]
                v = v-special.binom(k,i)*v2
            SKL[k][l] = v/wders[0][0]
    return SKL

## 2nd Order  element p=1, q=1

## Higher (4th) Order  element p=3, q=3

In [10]:
# #----------Manually change below variables--------------#
# p=3
# q=3
# U=np.array([0., 0., 0., 0., 1., 2., 2., 2., 2.])
# V=np.array([0., 0., 0., 0., 1., 2., 2., 2., 2.])
# P_W=np.array([[[0,0,0,1],[0,0.25,0,1],[0,0.5,0,1],[0,0.75,0,1],[0,1,0,1]],
#               [[0.25,0,0,1],[0.25,0.25,0,1],[0.25,0.5,0,1],[0.25,0.75,0,1],[0.25,1,0,1]],
#               [[0.5,0,0,1],[0.5,0.25,0,1],[0.5,0.5,0,1],[0.5,0.75,0,1],[0.5,1,0,1]],
#               [[0.75,0,0,1],[0.75,0.25,0,1],[0.75,0.5,0,1],[0.75,0.75,0,1],[0.75,1,0,1]],
#               [[1.0,0,0,1],[1.0,0.25,0,1],[1.0,0.5,0,1],[1.0,0.75,0,1],[1.0,1,0,1]]])
# d=2
# ndof = 2
# #-----------GAUSS POINTS---------------#
# GPs_Ws = np.array([[-0.8611,-0.8611,0.1210],[-0.8611,-0.3399,0.2269],[-0.8611,0.3399,0.2269],[-0.8611,0.8611,0.1210],
#                    [-0.3399,-0.8611,0.2269],[-0.3399,-0.3399,0.4253],[-0.3399,0.3399,0.4253],[-0.3399,0.8611,0.2269],
#                    [0.3399,-0.8611,0.2269],[0.3399,-0.3399,0.4253],[0.3399,0.3399,0.4253],[0.3399,0.8611,0.2269],
#                    [0.8611,-0.8611,0.1210],[0.8611,-0.3399,0.2269],[0.8611,0.3399,0.2269],[0.8611,0.8611,0.1210]])
#----------For Verification with 2nd order equations------------#
p=1
q=1
U = np.array([0., 0., 1.,1.]) 
V = np.array([0., 0., 1.,1.]) 
P_W=np.array([[[0,0,0,1],[0,1,0,1]],
           [[1,0,0,1],[1,1,0,1]]])
d=1
ndof = 2
#-----------GAUSS POINTS---------------#
GPs_Ws = np.array([[-0.5774,-0.5774,1],[0.5774,-0.5774,1],[0.5774,0.5774,1],[-0.5774,0.5774,1]])
#---------------------------------------------------------------#


#----------Need not alter below variables--------------#
n=(np.size(U)-1)-p-1
m=(np.size(V)-1)-q-1
P=P_W[:,:,0:3]
W=P_W[:,:,3]
#******Check how to dicide points in x and y direction*****************
ncpxi=np.shape(P_W)[1]  #No.of control points xi direction
ncpeta=np.shape(P_W)[0] #No.of control points eta direction
#**********************************************************************
#nel=(ncpxi-p-1)*(ncpeta-q-1)
ncp = ncpxi*ncpeta #Total number of control points
#necp = (p+2)*(q+2) #Total number of control points per element
nel=1
necp=4
print(np.shape(P_W)[0])

2


## Import to Material properties

In [11]:
yield_stress=210 #MPa
youngs_modulus=210000 #N/mm^2 #MPa
poissons_ratio=0.30
MU=(Youngs_modulus/(2*(1+Poissons_ratio)))

## Material Routine - Linear Elastic

In [ ]:
#????   from material_parameters import *  ????#

def materialRoutine(epsilon, T_m):
    mu = youngs_modulus/(2*(1+poissons_ratio))
    lamda = (poissons_ratio*youngs_modulus)/((1+poissons_ratio)*(1-2*poissons_ratio))
    sigma_y = yield_stress
    # Calculating Material tangent stiffness matrix
    Ct = np.array([[2*MU+lamda,lamda,0],[lamda,2*MU+lamda,0],[0,0,MU]]) 
    sigma_updated  = np.matmul(C_el,epsilon) 
    # returning Material tangent stiffness matrix and Updated stress to element routine
    return Ct, sigma_updated    

## Import to Element routine

In [ ]:
# #----------Manually change below variables--------------#
GPs_Ws = np.array([[-0.5774,-0.5774,1],[0.5774,-0.5774,1],[0.5774,0.5774,1],[-0.5774,0.5774,1]])
gauss_points = P_W[:,:,0:3]
gauss_weights = P_W[:,:,3]
p=1
q=1
U = np.array([0., 0., 1.,1.]) 
V = np.array([0., 0., 1.,1.]) 
P_W=np.array([[[0,0,0,1],[0,1,0,1]],
           [[1,0,0,1],[1,1,0,1]]])
d=1
ndof = 2
#----------Need not alter below variables--------------#
n=(np.size(U)-1)-p-1
m=(np.size(V)-1)-q-1
P=P_W[:,:,0:3]
W=P_W[:,:,3]
#******Check how to dicide points in x and y direction*****************
ncpxi=np.shape(P_W)[1]  #No.of control points xi direction
ncpeta=np.shape(P_W)[0] #No.of control points eta direction
#**********************************************************************
#nel=(ncpxi-p-1)*(ncpeta-q-1)
ncp = ncpxi*ncpeta #Total number of control points
#necp = (p+2)*(q+2) #Total number of control points per element
nel=1
necp=4

## Element Routine in progress

In [ ]:
#????    from Material_Routine import materialRoutine    ????#


def elementRoutine(U_e, T_m, X_e):

    for j in range(np.shape(GPs_Ws)[0]):
        gp = GPs_Ws[j,0:2]
        wg = GPs_Ws[j,2]
        ximas = gp[0]
        etamas = gp[1]
        xi = 0.5*((elU[1]-elU[0])*ximas + (elU[1]+elU[0]))     #Paramteric co-ordinates
        eta = 0.5*((elV[1]-elV[0])*etamas + (elV[1]+elV[0]))
        dxi_dximas = 0.5*(elU[1]-elU[0])
        deta_detamas = 0.5*(elV[1]-elV[0]) 
        J2 = dxi_dximas*deta_detamas

        #--------Evaluation of NURBS basis functions derivatives---------#
        Aders = SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,xi,eta,d)
        wders = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,xi,eta,d)
        dRx = RatSurfaceDerivs(Aders,wders,d)
        dRx_dxi  =  dRx[1][0][0]
        dRx_deta =  dRx[0][1][0]
        dRy_dxi  =  dRx[1][0][1]
        dRy_deta =  dRx[0][1][1]
        J1 = np.array([[dRx_dxi,dRx_deta],
                  [dRy_dxi,dRy_deta]])
        #print('J1 Matrix : ',J1)
        J1det = (dRx_dxi*dRy_deta)-(dRx_deta*dRy_dxi)
        #print('Determinat of J1 : ',J1det)
        J1inv = np.linalg.inv(J1)
        #print('J1inv : ',J1inv)
        #---------------NURBS Basis Functions Derivatives wrt x and y-------------#
        uspan = FindSpan(n,p,xi,U)
        vspan = FindSpan(m,q,eta,V)
        # Indices of non vanishing basis functions
        NVu = np.arange(uspan-p,uspan+1,1) #Thoroughly checked. Can crosscheck again
        NVv = np.arange(vspan-p,vspan+1,1)
        Den = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,xi,eta,d)
        Denom = Den[0][0]
        Denom_du = Den[1][0]
        Denom_dv = Den[0][1]
        dR_du = np.zeros((ncpxi,ncpeta))
        dR_dv = np.zeros((ncpxi,ncpeta))
        #---------------Loop over Non vanishing NURBS Basis Functions-------------#
        #***************Forgot to multiply with control points weights*************# Have to do it
        for ii, ii_value in enumerate(NVu):
            for jj, jj_value in enumerate(NVv):
                BFu = BasisFuns(uspan,xi,p,U)
                BFv = BasisFuns(vspan,eta,q,V)
                Num = BFu[ii]*BFv[jj]
                DBFu = DersBasisFuns(uspan,xi,p,d,U)
                DBFv = DersBasisFuns(vspan,eta,q,d,V)
                Num_du = DBFu[1][ii]*BFv[jj]
                Num_dv = BFu[ii]*DBFv[1][jj]
                dR_du[ii_value][jj_value] = Num_du/Denom - Denom_du*Num/(Denom*Denom)
                dR_dv[ii_value][jj_value] = Num_dv/Denom - Denom_dv*Num/(Denom*Denom)
        #---------Flatten (Convert 2D to 1D array) DervsNURBS Function------------#
        #dR_du(0,0)....dR_du(0,1),dR_du(1,0),.....dR_du(1,4).....dR_du(4,0),..........dR_du(4,4)
        #print(dR_du)
        #fdR_du = dR_du.flatten()
        fdR_du = (np.transpose(dR_du)).flatten() #************Cross check this******************#
        #print(fdR_du)
        #fdR_dv = dR_dv.flatten()
        fdR_dv = (np.transpose(dR_dv)).flatten()
        #print(fdR_dv)
        #-------------------------B Matrix --------------------------#
        for i2 in range(necp):
            j1= 2*i2
            j2= 2*i2+1
            B[0,j1] = fdR_du[i2]
            #print(j1,j2)
            #print(B[0,j1])
            B[1,j2] = fdR_dv[i2]
            #print(B[1,j2])
            B[2,j1] = fdR_dv[i2]
            #print(B[2,j1])
            B[2,j2] = fdR_du[i2]
            #print(B[2,j2])
        #print(B)
        epsilon = np.matmul(B,U_e)
        #*call element routine
        #------------------------- C Matrix--------------------------#
        C=np.array([[2*MU+lamda,lamda,0],[lamda,2*MU+lamda,0],[0,0,MU]])
        sigma = np.matmul(C,epsilon)
        #print(C)
        #C=np.array([[139000,74280,0],[74280,115400,0],[0,0,115400]])
        #-------------------------Local Stiffness matrix Ke-------------------#
        CB=np.matmul(C,B)
        BCB = np.matmul(np.transpose(B),CB)
        Kt_e = Kt_e + BCB*J1det*J2*wg
        F_int_e= F_int_e+np.matmul(np.transpose(B),sigma)*J1det*J2*wg
        F_ext_e = np.zeros_like(F_int_e)
        
    return Kt_e, F_int_e, F_ext_e,sigma,r  # What is r here??????

## Main Program

## Element Routine    ----------Have to seperate Nurbs derivatives--------

In [16]:
for i in range(nel):
    Ke=np.zeros((ndof*necp,ndof*necp))
    F_int_e = np.zeros((ndof*necp,1))
    B=np.zeros((3,ndof*necp)) #Dimentions depends on type of application
    elU = np.array([0,1]) #Manually defined have to generate using connectivity functions
    elV = np.array([0,1]) #Manually defined have to generate using connectivity functions
    
    #-------------------GAUSS LOOP-------------------#
    for j in range(np.shape(GPs_Ws)[0]):
    #for j in range(1):
        gp = GPs_Ws[j,0:2]
        wg = GPs_Ws[j,2]
        ximas = gp[0]
        etamas = gp[1]
        xi = 0.5*((elU[1]-elU[0])*ximas + (elU[1]+elU[0]))     #Paramteric co-ordinates
        eta = 0.5*((elV[1]-elV[0])*etamas + (elV[1]+elV[0]))
        dxi_dximas = 0.5*(elU[1]-elU[0])
        deta_detamas = 0.5*(elV[1]-elV[0]) 
        J2 = dxi_dximas*deta_detamas
        #print(J2)
        #--------Evaluation of NURBS basis functions derivatives---------#
        Aders = SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,xi,eta,d)
        wders = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,xi,eta,d)
        dRx = RatSurfaceDerivs(Aders,wders,d)
        dRx_dxi  =  dRx[1][0][0]
        dRx_deta =  dRx[0][1][0]
        dRy_dxi  =  dRx[1][0][1]
        dRy_deta =  dRx[0][1][1]
        J1 = np.array([[dRx_dxi,dRx_deta],
                  [dRy_dxi,dRy_deta]])
        #print('J1 Matrix : ',J1)
        J1det = (dRx_dxi*dRy_deta)-(dRx_deta*dRy_dxi)
        #print('Determinat of J1 : ',J1det)
        J1inv = np.linalg.inv(J1)
        #print('J1inv : ',J1inv)
        #---------------NURBS Basis Functions Derivatives wrt x and y-------------#
        uspan = FindSpan(n,p,xi,U)
        vspan = FindSpan(m,q,eta,V)
        # Indices of non vanishing basis functions
        NVu = np.arange(uspan-p,uspan+1,1) #Thoroughly checked. Can crosscheck again
        NVv = np.arange(vspan-p,vspan+1,1)
        Den = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,xi,eta,d)
        Denom = Den[0][0]
        Denom_du = Den[1][0]
        Denom_dv = Den[0][1]
        dR_du = np.zeros((ncpxi,ncpeta))
        dR_dv = np.zeros((ncpxi,ncpeta))
        #---------------Loop over Non vanishing NURBS Basis Functions-------------#
        #***************Forgot to multiply with control points weights*************# Have to do it
        for ii, ii_value in enumerate(NVu):
            for jj, jj_value in enumerate(NVv):
                BFu = BasisFuns(uspan,xi,p,U)
                BFv = BasisFuns(vspan,eta,q,V)
                Num = BFu[ii]*BFv[jj]
                DBFu = DersBasisFuns(uspan,xi,p,d,U)
                DBFv = DersBasisFuns(vspan,eta,q,d,V)
                Num_du = DBFu[1][ii]*BFv[jj]
                Num_dv = BFu[ii]*DBFv[1][jj]
                dR_du[ii_value][jj_value] = Num_du/Denom - Denom_du*Num/(Denom*Denom)
                dR_dv[ii_value][jj_value] = Num_dv/Denom - Denom_dv*Num/(Denom*Denom)
        #---------Flatten (Convert 2D to 1D array) DervsNURBS Function------------#
        #dR_du(0,0)....dR_du(0,1),dR_du(1,0),.....dR_du(1,4).....dR_du(4,0),..........dR_du(4,4)
        #print(dR_du)
        #fdR_du = dR_du.flatten()
        fdR_du = (np.transpose(dR_du)).flatten() #************Cross check this******************#
        #print(fdR_du)
        #fdR_dv = dR_dv.flatten()
        fdR_dv = (np.transpose(dR_dv)).flatten()
        #print(fdR_dv)
        #-------------------------B Matrix --------------------------#
        for i2 in range(necp):
            j1= 2*i2
            j2= 2*i2+1
            B[0,j1] = fdR_du[i2]
            #print(j1,j2)
            #print(B[0,j1])
            B[1,j2] = fdR_dv[i2]
            #print(B[1,j2])
            B[2,j1] = fdR_dv[i2]
            #print(B[2,j1])
            B[2,j2] = fdR_du[i2]
            #print(B[2,j2])
        #print(B)
        #***epsilon = np.matmul(B,U_e)
        #*call element routine
        #------------------------- C Matrix--------------------------#
        C=np.array([[2*MU+lamda,lamda,0],[lamda,2*MU+lamda,0],[0,0,MU]])
        sigma = np.matmul(C,epsilon)
        #print(C)
        #C=np.array([[139000,74280,0],[74280,115400,0],[0,0,115400]])
        #-------------------------Local Stiffness matrix Ke-------------------#
        CB=np.matmul(C,B)
        BCB = np.matmul(np.transpose(B),CB)
        Ke = Ke + BCB*J1det*J2*wg
        F_int_e= F_int_e+np.matmul(np.transpose(B),sigma)*J1det*J2*wg
    #print(np.linalg.det(Ke)) 
    #print(Ke)
        

NameError: name 'epsilon' is not defined

## Main Program

In [14]:
#-----------Import these functions later----------# 
#from mesh_generation import *
#from material_parameters import *
#from assignment_matrix import ASSIGNMENT_MATRIX
#from Element_Routine import elementRoutine
#-------------------------------------------------#

#initialization of time like parameter and displacement vector and state variables
allTimes = np.arange(tau_start+delta_t, tau_end+delta_t, delta_t)
allTimes[-1] = tau_end
nSteps = len(allTimes)


#Initializing global displcement vector and stress vector 
#nnodes=Total number of nodes ndofpn = number of Dof per node 
U_g_0=np.zeros((nnodes*ndofpn,1))
global_sigma = np.zeros([nSteps,nElem,3,1])


#loop for iterating load from 0% to 100%

for i in range(nSteps):
    tau = allTimes[i] 
    u_g = U_g_0 
    #u_g[0,0] = 1/3*E_v*tau*rnodes[0]
    #current_E_pl = np.zeros_like(E_pls)
    current_sigma = np.zeros([nElem,3,1])
    
#------------------DO Newton_Raphson_method----------------------------#
    k=1
    while 1:
        #gauss_loc = np.zeros(nElem)
        Kt_g=np.zeros([nnodes*ndofpn,nnodes*ndofpn])
        G_global=np.zeros((nnodes*ndofpn,1))
        F_g_int=np.zeros((nnodes*ndofpn,1))
        F_g_ext=np.zeros((nnodes*ndofpn,1))
        
#------------------------Connectivity loop------------------------------#
#         for j in range(nElem):
#             A=ASSIGNMENT_MATRIX(j+1,nElem)
#             u_e=np.matmul(A,u_g)
#             # calling Element routine
#             K_e,F_e_int,F_e_ext,E_pl, sigma, r_gp = elementRoutine(u_e,tau,rnodes[j:j+2],E_pls[j])
#             Kt_g = Kt_g+np.matmul(np.transpose(A),np.matmul(K_e,A))
#             G_global = G_global+np.matmul(np.transpose(A),(F_e_int-F_e_ext))
#             F_g_int = F_g_int+np.matmul(np.transpose(A),(F_e_int))
#             current_E_pl[j] = E_pl
#             current_sigma[j] = sigma        
#             gauss_loc[j] = r_gp
#------------------------------------------------------------------------#
        for j in range(nElem):
            #nnodes_e  = number of nodes per element
            u_e = np.zeros((nnodes_e,1))
            #** Call element Routine
            Kt_g = Kt_g+Ke
            G_global = G_global+(F_g_int-F_g_ext)
            F_g_int = 
        #Reduced system of equations
        K_rg=Kt_g
        K_rg=np.delete(K_rg,0,axis=0)
        K_rg=np.delete(K_rg,0,axis=1)
        reduced_G_global=G_global
        reduced_G_global=np.delete(reduced_G_global,0,axis=0)
        dU_g=np.matmul(np.linalg.inv(K_rg),-reduced_G_global)
        u_g[1:]=u_g[1:]+dU_g

        if (np.linalg.norm(reduced_G_global,np.inf)<0.005*np.linalg.norm(F_g_int,np.inf) or np.linalg.norm(dU_g,np.inf)<0.005*np.linalg.norm(u_g[1:],np.inf)) or k > 5 :     
            break
        else:
            k=k+1
    if k>5:
        print("Convergence criterion is not met, at load (peercentage):",tau*100)
        break
    else:
        U_g_0 = u_g
        E_pls=current_E_pl
        global_sigma[i] = current_sigma
